In [6]:
# !export https_proxy=http://127.0.0.1:7890 http_proxy=http://127.0.0.1:7890 all_proxy=socks5://127.0.0.1:7890
# %pip install pandas
# %pip install pymysql

In [23]:
from flask import Flask
from flask_sqlalchemy import SQLAlchemy
import pandas as pd
from datetime import datetime

import pymysql
pymysql.install_as_MySQLdb()

In [28]:
# 配置
HOSTNAME = "127.0.0.1"
PORT = "3306"
USERNAME = "root"
PASSWORD = "wangjin"
DATABASE = "notice"

In [29]:
app = Flask(__name__)
# 使用连接信息进行替换
app.config['SQLALCHEMY_DATABASE_URI'] = f"mysql+pymysql://{USERNAME}:{PASSWORD}@{HOSTNAME}:{PORT}/{DATABASE}?charset=utf8mb4"
db = SQLAlchemy(app)

In [34]:
import sqlalchemy
# 检查数据库是否链接成功
with app.app_context():
    with db.engine.connect() as conn:
        rs = conn.execute(sqlalchemy.text("select 1"))
        print(rs.fetchone())  # 链接成功，返回(1,)


(1,)


In [39]:
# NotificationModel定义
class NotificationModel(db.Model):
    __tablename__ = "notification_data"
    
    id = db.Column(db.Integer, primary_key=True, autoincrement=True)
    title = db.Column(db.String(200), nullable=False)
    content = db.Column(db.Text, nullable=False)
    # 提取日期
    date = db.Column(db.DateTime, nullable=True)
    # 摘要
    summary = db.Column(db.Text, nullable=True)
    creator_id = db.Column(db.Integer)
    created_date = db.Column(db.DateTime, default=datetime.now)

/home/wangfiox/.pyenv/versions/anaconda3-2023.09-0/lib/python3.11/site-packages/flask_sqlalchemy/model.py:144: SAWarning: This declarative base already contains a class with the same class name and module name as __main__.NotificationModel, and will be replaced in the string-lookup table.
  super().__init__(name, bases, d, **kwargs)


InvalidRequestError: Table 'notification_data' is already defined for this MetaData instance.  Specify 'extend_existing=True' to redefine options and columns on an existing Table object.

In [40]:
def load_csv_into_db(filename):
    # 读取CSV
    df = pd.read_csv(filename)
    with app.app_context():
        # 遍历每一行并插入到数据库中
        for _, row in df.iterrows():
            # 解析日期
            date_str = row['日期']
            # 提取日期
            if isinstance(date_str, str):
                date = datetime.strptime(date_str, "%Y年%m月%d日") if "年" in date_str else pd.to_datetime(date_str)
            else:
                date = None  # 或者设置为其他默认值
            
            notification = NotificationModel(
                title=row['类别'],
                content=row['通知内容'],
                date=date,
                summary=row['概括内容'],
                creator_id=1
            )
            db.session.add(notification)
        db.session.commit()

# 加载CSV到数据库
load_csv_into_db("./dates_combined_data_summarized.csv")

DataError: (pymysql.err.DataError) (1406, "Data too long for column 'content' at row 1")
[SQL: INSERT INTO notification_data (title, content, date, summary, creator_id, created_date) VALUES (%(title__0)s, %(content__0)s, %(date__0)s, %(summary__0)s, %(creator_id__0)s, %(created_date__0)s), (%(title__1)s, %(content__1)s, %(date__1)s, %(summary_ ... 33384 characters truncated ... ator_id__302)s, %(created_date__302)s) RETURNING notification_data.id, notification_data.id AS id__1]
[parameters: {'content__0': '【哈工大卓越工程师学院2024年硕士研究生招生宣讲】@所有人\n各位同学，卓越工程师之类的项目是不严格要求保研名额的，有保研资格都可以报名（即前50%），请感兴趣的同学参会学习了解主讲嘉宾：哈工大卓越工程师学院党委书记兼常务副院长潘旭东教授宣讲时间：7月28日（周五14:30-16:00腾讯会议： ... (56 characters truncated) ... .cn/2023/0724/c15565a321323/page.htm卓越工程师学院2024年硕士研究生招生各专项简介http://ees.hit.edu.cn/2023/0720/c15565a321199/page.htm请各位同学参加了解。如有招生相关问题，请联系0451-86402619', 'summary__0': '哈工大卓越工程师学院2024年硕士研究生招生宣讲', 'creator_id__0': 1, 'title__0': '升学', 'date__0': None, 'created_date__0': datetime.datetime(2023, 12, 15, 12, 